In [1]:
#import packages and functions

%load_ext autoreload
%autoreload 2

import pursuit_functions as pursuit
    
import pandas as pd
import numpy as np
import polars as pl
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 

from scipy import stats
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
#load data set

all_pursuit_tasks = pd.read_parquet("ca1_ca3_rsc_pursuit_data.parquet", engine="pyarrow")

# Normalize points and find circle boundaries.

In [3]:
#get all coordinate values below 99th percentile and normalize points for all regions 

normalized_sessions = pursuit.tuning.normalize_points(all_pursuit_tasks)

In [ ]:
normalized_sessions.head()

In [4]:
#find the mean center and overall radius of the arena for all normalized data points
#you can specify the percentile value to be considered for the overall radius; default is 95th percentile
#calculates the individual center point for each session

circle_boundaries, radius = pursuit.tuning.fit_circle_bounds(normalized_sessions)
print(radius)

61.179933135984626


In [ ]:
circle_boundaries.head()

In [ ]:
#find circumference points for plotting using the center coordinates and overall radius
all_circ_points = pursuit.tuning.circumference(circle_boundaries)

# Plot the laser coordinates and boundaries.

In [ ]:
#plot normalized concatenated laser and rat paths with center point and boundary
#the function takes the normalized_sessions, circle_boundaries, and all_circ_points dataframes

pursuit.tuning.plot_arena_bounds(normalized_sessions, circle_boundaries, all_circ_points)

# Clean data and pull spike data.

In [5]:
# obtain region-specific sessions
RSC_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "RSC"]
CA1_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "CA1"]
CA3_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "CA3"]



In [6]:
#obtain trial block-specific sessions

RSC_pursuit = RSC_sessions[RSC_sessions["trial_block"] == "pursuit"]
CA1_pursuit = CA1_sessions[CA1_sessions["trial_block"] == "pursuit"]
CA3_pursuit = CA3_sessions[CA3_sessions["trial_block"] == "pursuit"]

In [7]:
#drop NA values for RSC, CA1, and CA3 sessions

RSC_cleaned = pursuit.tuning.drop_NA_vals(RSC_pursuit)
CA1_cleaned = pursuit.tuning.drop_NA_vals(CA1_pursuit)
CA3_cleaned = pursuit.tuning.drop_NA_vals(CA3_pursuit)

In [8]:
# there's some data compression that collapses the 120hz recording time points as the recordings get longer. 
# first we collapse the time points to whole seconds
# we need to normalize the time so that the first observation starts from 0 seconds
# we then calculate the normalized minute each observation belongs to

def normalize_time(dataframe):
    df = dataframe.copy()
    df["time"] = df["time"].astype(float)
    df["relative_time"] = df.groupby("sessFile")["time"].transform(lambda x: x - x.min())
    df["norm_sec"] = df["relative_time"].astype(int)
    df["norm_min"] = df["norm_sec"] // 60

    return df


In [9]:
RSC_clean_time = normalize_time(RSC_cleaned)

In [ ]:
RSC_clean_time

In [10]:
# to split each session into epochs, we separate them by first/second half of the recording or odd/even minutes
# for epoch_half, True:1, False:2 will result in the first half labeled as 1 (<= cutoff) and the second half labeled as 2
# for epoch_odd_even, we take the minutes divided by 2 and find the remainder. If the remainder is 1 (odd), it will be labeled as 1 and if the remainder is 0 (even), it will be labeled as 2

def assign_epochs(dataframe):
    df = dataframe.copy()

    # separate epochs by half

    def label_half(group):
        mins = group["norm_min"].unique()
        mins.sort()
        cutoff = mins[len(mins) // 2]
        return group["norm_min"] <= cutoff
    
    df["epoch_half"] = (
        df.groupby("sessFile", group_keys=False)
        .apply(label_half, include_groups=False)
        .map({True: 1, False: 2})
    )

    # separate epochs by odd/even minutes

    df["epoch_odd_even"] = df["norm_min"] % 2
    df["epoch_odd_even"] = df["epoch_odd_even"].map({1: 1, 0: 2})

    return df


In [11]:
# for epoch_half, True:1, False:2 will result in the first half labeled as 1 (<= cutoff) and the second half labeled as 2
# for epoch_odd_even, we take the minutes divided by 2 and find the remainder. If the remainder is 1 (odd), it will be labeled as 1 and if the remainder is 0 (even), it will be labeled as 2

RSC_clean_time_epochs = assign_epochs(RSC_clean_time)

RSC_clean_time_epochs

,time,ind,ratPos_1,ratPos_2,laserPos_1,laserPos_2,ratVel,ratAcc,laserVel,laserAcc,...,spkTable_33,spkTable_34,region,trial_block,sessFile,relative_time,norm_sec,norm_min,epoch_half,epoch_odd_even
7080778,138.625,8321,161.75,90.0625,146.0,76.3125,43.375,60.8125,36.46875,79.1875,...,<NA>,<NA>,RSC,pursuit,KB10_02_pursuitRoot.mat,0.000,0,0,1,2
7080779,138.750,8322,161.25,89.4375,145.5,76.125,44.40625,59.09375,37.78125,78.625,...,<NA>,<NA>,RSC,pursuit,KB10_02_pursuitRoot.mat,0.125,0,0,1,2
7080780,138.750,8323,160.75,88.6875,145.0,76.0625,45.375,57.46875,39.09375,78.1875,...,<NA>,<NA>,RSC,pursuit,KB10_02_pursuitRoot.mat,0.125,0,0,1,2
7080781,138.750,8324,160.25,88.0,144.5,75.9375,46.3125,56.125,40.40625,77.8125,...,<NA>,<NA>,RSC,pursuit,KB10_02_pursuitRoot.mat,0.125,0,0,1,2
7080782,138.750,8325,159.875,87.25,144.0,75.8125,47.1875,55.0625,41.6875,77.4375,...,<NA>,<NA>,RSC,pursuit,KB10_02_pursuitRoot.mat,0.125,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49368102,2172.000,130295,91.6875,111.4375,99.6875,118.625,67.125,89.8125,25.296875,586.5,...,<NA>,<NA>,RSC,pursuit,LP06_29_pursuitRoot.mat,1478.500,1478,24,2,2
49368103,2172.000,130296,91.9375,112.5,99.625,119.125,66.3125,88.8125,25.015625,652.0,...,<NA>,<NA>,RSC,pursuit,LP06_29_pursuitRoot.mat,1478.500,1478,24,2,2
49368104,2172.000,130297,92.25,113.625,99.5,119.75,65.4375,87.9375,28.859375,721.0,...,<NA>,<NA>,RSC,pursuit,LP06_29_pursuitRoot.mat,1478.500,1478,24,2,2
49368105,2172.000,130298,92.5625,114.6875,99.25,120.6875,64.5,87.3125,36.59375,791.5,...,<NA>,<NA>,RSC,pursuit,LP06_29_pursuitRoot.mat,1478.500,1478,24,2,2


In [ ]:
RSC_clean_time_epochs[RSC_clean_time_epochs["sessFile"] == "KB10_02_pursuitRoot.mat"]

In [12]:
#make a new df with only sessFile, laser, epoch, and spike data
def epoch_laser_spks(dataframe, laser_x="laserPos_1", laser_y="laserPos_2"):

    epoch_laser_spks_data = []

    spk_columns = [col for col in dataframe.columns if "spkTable" in col]

    time_column = [col for col in dataframe.columns if "relative_time" in col.lower()]
    
    for sessFile in dataframe["sessFile"].unique():

            session = dataframe[dataframe["sessFile"] == sessFile].copy()
            
            laser_x_vals = session[laser_x].astype("float64")
            laser_y_vals = session[laser_y].astype("float64")
    
            #identify 99th percentile x, y boundaries
            x_low, x_high = np.percentile(laser_x_vals, [0, 99])
            y_low, y_high = np.percentile(laser_y_vals, [0, 99])

            #filter the data so we only get the data under the 99th percentile
            filter = (
                (laser_x_vals >= x_low) & (laser_x_vals <= x_high) & 
                (laser_y_vals >= y_low) & (laser_y_vals <= y_high)
            )

            filtered_session = session[filter].copy()

            #normalize the points to the origin
            x_normalized = filtered_session[laser_x].astype("float64") - float(x_low)
            y_normalized = filtered_session[laser_y].astype("float64") - float(y_low)

            #grab epoch data 
            epoch_half = filtered_session["epoch_half"].values
            epoch_odd_even = filtered_session["epoch_odd_even"].values

            #make a dataframe containing normalized data
            normalized_df = pd.DataFrame({
                "sessFile": sessFile,
                "laser_x_normalized": x_normalized.values,
                "laser_y_normalized": y_normalized.values,
                "epoch_half": epoch_half,
                "epoch_odd_even": epoch_odd_even
            })

            #grab spike data using the normalized data mask
            spk_df = filtered_session[spk_columns].reset_index(drop=True)

            #grab time data using the normalized data mask
            time_df = filtered_session[time_column].reset_index(drop=True)

            #make a combined dataframe
            combined_df = pd.concat([normalized_df.reset_index(drop=True), spk_df, time_df], axis=1)

            #append dataframe to the list
            epoch_laser_spks_data.append(combined_df)

        #make a giant dataframe by concatenating all the dataframes in the list        
    epoch_laser_spks_df = pd.concat(epoch_laser_spks_data, ignore_index=True)

    return epoch_laser_spks_df

In [13]:
RSC_epoch_laser_spks = epoch_laser_spks(RSC_clean_time_epochs)

In [ ]:
RSC_epoch_laser_spks[RSC_epoch_laser_spks["sessFile"] == "KB10_02_pursuitRoot.mat"]

In [14]:
#function for pulling epochs from each session along with associated sessFile, laser, and spike data

def pull_epochs(dataframe, 
                spk_prefix="spkTable"):

    epoch_first_half = []
    epoch_second_half = []
    epoch_odd_min = []
    epoch_even_min = []

    for sessFile in dataframe["sessFile"].unique():
        
        session = dataframe[dataframe["sessFile"] == sessFile].copy()

        # mapping first half and second epochs for each session 
        first_half = session[session["epoch_half"] == 1]
        second_half = session[session["epoch_half"] ==2]
        # mapping odd and even epochs for each session
        odd_min = session[session["epoch_odd_even"] == 1]
        even_min = session[session["epoch_odd_even"] == 2]

        spk_cols = [col for col in session.columns if spk_prefix in col and not session[col].isna().all()]

        #function for grabbing sessFile, laser x, laser y, and spk columns for each epoch
        def build_epoch_df(epoch):
            return pd.concat([
                epoch[["sessFile", "laser_x_normalized", "laser_y_normalized", "relative_time"]].reset_index(drop=True),
                epoch[spk_cols].reset_index(drop=True)
            ], axis=1)

        epoch_first_half.append(build_epoch_df(first_half))
        epoch_second_half.append(build_epoch_df(second_half))
        epoch_odd_min.append(build_epoch_df(odd_min))
        epoch_even_min.append(build_epoch_df(even_min))

    return (
        pd.concat(epoch_first_half, ignore_index=True),
        pd.concat(epoch_second_half, ignore_index=True),
        pd.concat(epoch_odd_min, ignore_index=True),
        pd.concat(epoch_even_min, ignore_index=True),
    )
    


In [15]:
RSC_epoch_first_half, RSC_epoch_second_half, RSC_epoch_odd_min, RSC_epoch_even_min = pull_epochs(RSC_epoch_laser_spks)

In [16]:
RSC_epoch_first_half

,sessFile,laser_x_normalized,laser_y_normalized,relative_time,spkTable_1,spkTable_2,spkTable_3,spkTable_4,spkTable_5,spkTable_6,...,spkTable_25,spkTable_26,spkTable_27,spkTable_28,spkTable_29,spkTable_30,spkTable_31,spkTable_32,spkTable_33,spkTable_34
0,KB10_02_pursuitRoot.mat,69.8125,62.851562,0.000,0,0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,KB10_02_pursuitRoot.mat,69.3125,62.664062,0.125,0,0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,KB10_02_pursuitRoot.mat,68.8125,62.601562,0.125,1,0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,KB10_02_pursuitRoot.mat,68.3125,62.476562,0.125,0,1,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,KB10_02_pursuitRoot.mat,67.8125,62.351562,0.125,0,0,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3372751,LP06_29_pursuitRoot.mat,91.0000,50.390625,779.500,0,0,0,0,0,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3372752,LP06_29_pursuitRoot.mat,91.6250,51.390625,779.500,0,0,0,0,0,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3372753,LP06_29_pursuitRoot.mat,92.2500,52.390625,779.500,0,0,0,0,1,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3372754,LP06_29_pursuitRoot.mat,92.7500,53.265625,779.500,0,0,0,0,0,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [17]:
# boostrapping function calculates the true corr then uses the relative time to shift the spktables before calculating the distance, bin spks/laser vals, and calculating tuning

def make_assignment_matrix(bin_assignments, dist_bin_edges):
    bin_ids = np.digitize(bin_assignments, dist_bin_edges, right=False) - 1
    bin_mask = (bin_ids >= 0) & (bin_ids < len(dist_bin_edges) -1)

    B = len(dist_bin_edges) - 1
    T = len(bin_assignments)
    M = np.zeros((B, T))
    np.add.at(M, bin_ids[bin_mask], 1)
    return M


In [18]:
def tuning_calc(M, spk_array):
    spk_sum = M @ spk_array # (B, N)
    occupancy = M.sum(axis=1) # (B,)
    tuning = np.divide(spk_sum, occupancy[:, None], where=occupancy[:, None] != 0)
    return tuning, occupancy

In [19]:
def make_shift_idx(rel_time, num_shifts=10):
    rel_time_np = rel_time.to_numpy().flatten()
    shift_points = np.where(np.diff(np.floor(rel_time_np)) > 0)[0] + 1
    return shift_points[:num_shifts].tolist()

In [20]:
def bootstrap_tuning_corr(
        epoch_df1: pd.DataFrame, 
        epoch_df2: pd.DataFrame, 
        center_df: pd.DataFrame, 
        spk_prefix: str = "spkTable", 
        rel_time_col: str = "relative_time", 
        num_shifts: int = 10
        ) -> pd.DataFrame:
    all_results = []

    spk_cols = [col for col in epoch_df1.columns if spk_prefix in col]
    sessions = epoch_df1["sessFile"].unique()

    lf1 = pl.LazyFrame(epoch_df1)
    lf2 = pl.LazyFrame(epoch_df2)

    for sessFile in tqdm(sessions, desc="Sessions"):
        df1 = lf1.filter(pl.col("sessFile") == sessFile).collect().to_pandas()
        df2 = lf2.filter(pl.col("sessFile") == sessFile).collect().to_pandas()

        # calculate distance to bounds
        dist1 = pursuit.tuning.dist_to_bounds(df1, center_df)["bound_dist"].values
        dist2 = pursuit.tuning.dist_to_bounds(df2, center_df)["bound_dist"].values

        # get spike arrays
        spk1 = df1[spk_cols].to_numpy()
        spk2 = df2[spk_cols].to_numpy()

        # get bin edges
        dist_bin_edges = np.linspace(min(dist1.min(), dist2.min()),
                                    max(dist1.max(), dist2.max()), 21)                                   

        # make bin assignment matrices 
        M1 = make_assignment_matrix(dist1, dist_bin_edges)
        M2 = make_assignment_matrix(dist2, dist_bin_edges)

        # calculate tuning
        tune1, _ = tuning_calc(M1, spk1)
        tune2, _ = tuning_calc(M2, spk2)

        # spearman corr- iter: 0 (True corr)
        for j, ncol in enumerate(spk_cols):
            r, p = stats.spearmanr(tune1[:, j], tune2[:, j])
            all_results.append({
                "sessFile": sessFile,
                "neuron": ncol,
                "bootstrap_iter": 0,
                "spearman_r": float(r),
                "p_val": float(p),
            })

        #bootstrapping with shift indices from lazyframe version of rel_time

        rel_time = df1[rel_time_col]
        shift_points = make_shift_idx(rel_time, num_shifts=num_shifts)

        for i, shift_idx in enumerate(shift_points, start=1):
            spk1_shifted = np.roll(spk1, shift_idx, axis=0) 
            tune1_rolled, _ = tuning_calc(M1, spk1_shifted)

            for j, ncol in enumerate(spk_cols):
                r, p = stats.spearmanr(tune1_rolled[:, j], tune2[:, j])
                all_results.append({
                    "sessFile": sessFile,
                    "neuron": ncol,
                    "bootstrap_iter": i,
                    "spearman_r": float(r),
                    "p_val": float(p),
                })

    return pd.DataFrame(all_results)

In [21]:
RSC_first_second_tuning = bootstrap_tuning_corr(RSC_epoch_first_half, RSC_epoch_second_half, circle_boundaries)

Sessions:   0%|          | 0/46 [00:00<?, ?it/s]/var/folders/k4/zr04khhn74zcj29mf4f0v8hc0000gn/T/ipykernel_64441/1110672188.py:43: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.spearmanr(tune1[:, j], tune2[:, j])
/var/folders/k4/zr04khhn74zcj29mf4f0v8hc0000gn/T/ipykernel_64441/1110672188.py:62: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.spearmanr(tune1_rolled[:, j], tune2[:, j])
Sessions:   2%|▏         | 1/46 [03:08<2:21:31, 188.69s/it]/var/folders/k4/zr04khhn74zcj29mf4f0v8hc0000gn/T/ipykernel_64441/1110672188.py:43: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.spearmanr(tune1[:, j], tune2[:, j])
/var/folders/k4/zr04khhn74zcj29mf4f0v8hc0000gn/T/ipykernel_64441/1110672188.py:62: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.spearmanr(

: 

: 

# ignore chunks below

In [ ]:
RSC_first_second_tuning 

In [ ]:
#tuning curves for first half and second half epochs
    

RSC_first_half_bounds = pursuit.tuning.dist_to_bounds(RSC_epoch_first_half, circle_boundaries)
RSC_first_half_binned = pursuit.tuning.bin_spikes_laser(RSC_first_half_bounds)
RSC_first_half_tuning = pursuit.tuning.calculate_tuning(RSC_first_half_binned)

RSC_second_half_bounds = pursuit.tuning.dist_to_bounds(RSC_epoch_second_half, circle_boundaries)
RSC_second_half_binned = pursuit.tuning.bin_spikes_laser(RSC_second_half_bounds)
RSC_second_half_tuning = pursuit.tuning.calculate_tuning(RSC_second_half_binned)



In [ ]:
#tuning curves for odd and even min epochs

RSC_odd_min_bounds = pursuit.tuning.dist_to_bounds(RSC_epoch_odd_min, circle_boundaries)
RSC_odd_min_binned = pursuit.tuning.bin_spikes_laser(RSC_odd_min_bounds)
RSC_odd_min_tuning = pursuit.tuning.calculate_tuning(RSC_odd_min_binned)

RSC_even_min_bounds = pursuit.tuning.dist_to_bounds(RSC_epoch_even_min, circle_boundaries)
RSC_even_min_binned = pursuit.tuning.bin_spikes_laser(RSC_even_min_bounds)
RSC_even_min_tuning = pursuit.tuning.calculate_tuning(RSC_even_min_binned)

In [ ]:
RSC_first_half_tuning

In [ ]:
RSC_second_half_tuning

In [ ]:
#merge epoch dfs with tuning values for spearman correlation

def merge_epoch_tuning(df_epoch1, df_epoch2, label1="epoch1", label2="epoch2"):
           
    df1 = df_epoch1.copy().rename(columns={"tuning": f"tuning_{label1}"})
    df2 = df_epoch2.copy().rename(columns={"tuning": f"tuning_{label2}"})

    merged_df = pd.merge(
        df1[["sessFile", "neuron", "bin_midpoint", f"tuning_{label1}"]],
        df2[["sessFile", "neuron", "bin_midpoint", f"tuning_{label2}"]],
        on=["sessFile", "neuron", "bin_midpoint"],
        how="inner"
    )

    return merged_df
    
    

In [ ]:
RSC_half_tuning = merge_epoch_tuning(RSC_first_half_tuning, RSC_second_half_tuning)

In [ ]:
RSC_half_tuning

In [ ]:
# function for calculating spearman correlations from tuning values

def spearman_corr(tuning_df):
        results = []
        for (sess, neuron), group in tuning_df.groupby(["sessFile", "neuron"]):
                r, p = stats.spearmanr(group["tuning_epoch1"], group["tuning_epoch2"])
                group = group.copy()
                group["spearman_r"] = r
                group["p_val"] = p
                results.append(group)

        return pd.concat(results, ignore_index=True)

In [ ]:
RSC_tuning_corr = spearman_corr(RSC_half_tuning)

In [ ]:
RSC_tuning_corr

In [ ]:

def spearman_corr_boot(dataframe1, dataframe2, bootstrap_iter):
    
    correlation_results = []
    
    group_1 = dataframe1.groupby(["sessFile", "neuron"])
    group_2 = dataframe2.groupby(["sessFile", "neuron"])

    common_keys = set(group_1.groups.keys()) & set(group_2.groups.keys())

    for key in sorted(common_keys):
        sessFile, neuron = key

        g1 = group_1.get_group(key).sort_values("bin_midpoint")
        g2 = group_2.get_group(key).sort_values("bin_midpoint")

        t1 = g1["tuning"].values
        t2 = g2["tuning"].values
        bins = g1["bin_midpoint"].values

        if len(t1) != len(t2):
            print(f"The lengths of the tuning curves are unequal for {key}!!!")
            continue

        corr_val, p_val = stats.spearmanr(t1, t2)
        
        correlation_results.append({
            "sessFile": sessFile,
            "neuron": neuron,
            "bin_midpoint": bins,
            "bootstrap_iter": bootstrap_iter,
            "spearman_r": corr_val,
            "p_value": p_val
        })

    return pd.DataFrame(correlation_results)

In [ ]:
#calculate the true tuning correlation

epoch_half_true_tuning_corr = spearman_corr_boot(RSC_first_half_tuning, RSC_second_half_tuning, bootstrap_iter=0)


In [ ]:
epoch_half_true_tuning_corr

In [ ]:
shift spikes circularly for bootstrapping 

In [ ]:
then make a loop to do it x1000

In [ ]:
def unique_time_entries(dataframe):
    all_sessions = []

    for sessFile in dataframe["sessFile"].unique():
        session = dataframe[dataframe["sessFile"] == sessFile]
        session_times = session["time"]

        times_unique = session_times.astype("float64").nunique()
        times_count = session_times.astype("float64").value_counts().to_dict()

        all_sessions.append({
            "sessFile": sessFile,
            "times_unique": times_unique,
            "unique_times_count": times_count
        })

    return pd.DataFrame(all_sessions)

In [ ]:
RSC_sessions_times = unique_time_entries(RSC_sessions)
RSC_sessions_times

In [ ]:
def inspect_time_counts(session_row):
    times= session_row["unique_times_count"]
    sorted_times = sorted(times.items())
    return pd.DataFrame(sorted_times, columns=["time", "count"])

In [ ]:
lp03_25 = RSC_sessions_times[RSC_sessions_times["sessFile"] == "LP03_25_pursuitRoot.mat"]

lp03_25_row = lp03_25.iloc[0]

lp03_25_inspection = inspect_time_counts(lp03_25_row)


lp03_25_inspection

In [ ]:
RSC_cleaned.head(10)

In [ ]:
#normalize only laser points and make a dataframe containing spike data using the normalized data mask
#function takes the cleaned df
RSC_laser_spks = pursuit.tuning.norm_laser_get_spks(RSC_cleaned)
CA1_laser_spks = pursuit.tuning.norm_laser_get_spks(CA1_cleaned)
CA3_laser_spks = pursuit.tuning.norm_laser_get_spks(CA3_cleaned)

In [ ]:
CA1_laser_spks.head(50)

# ignore chunks above

# Find distance of laser points to boundary and bin data by distance.

In [ ]:
#find distance of normalized laser points to circle boundary by each session
#function takes the normalized laser/spikes and circle boundaries dataframes

RSC_laser_spks_bounds = pursuit.tuning.dist_to_bounds(RSC_laser_spks, circle_boundaries)
CA1_laser_spks_bounds = pursuit.tuning.dist_to_bounds(CA1_laser_spks, circle_boundaries)
CA3_laser_spks_bounds = pursuit.tuning.dist_to_bounds(CA3_laser_spks, circle_boundaries)

In [ ]:
CA3_laser_spks_bounds.head(50)

In [ ]:
#bin the data!
RSC_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(RSC_laser_spks_bounds)
CA1_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(CA1_laser_spks_bounds)
CA3_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(CA3_laser_spks_bounds)

# Normalize spike counts, smooth data, peak sort neurons, and plot tuning curves.

In [ ]:
#calculate raw tuning curves
RSC_tuning = pursuit.tuning.calculate_tuning(RSC_laser_spikes_binned)
CA1_tuning = pursuit.tuning.calculate_tuning(CA1_laser_spikes_binned)
CA3_tuning = pursuit.tuning.calculate_tuning(CA3_laser_spikes_binned)

In [ ]:
#plot all neuron tuning curves for a sanity check

pursuit.tuning.plot_tuning_curves(CA3_tuning)

In [ ]:
# z-score and normalize data
RSC_z_scored = pursuit.tuning.z_score_norm(RSC_tuning)
CA1_z_scored = pursuit.tuning.z_score_norm(CA1_tuning)
CA3_z_scored = pursuit.tuning.z_score_norm(CA3_tuning)

In [ ]:
# apply smoothing and pivot data
RSC_smoothed = pursuit.tuning.pivot_smooth(RSC_z_scored)
CA1_smoothed = pursuit.tuning.pivot_smooth(CA1_z_scored)
CA3_smoothed = pursuit.tuning.pivot_smooth(CA3_z_scored)

In [ ]:
# peak sort the data
RSC_smoothed_sorted = pursuit.tuning.peak_sort(RSC_smoothed)
CA1_smoothed_sorted = pursuit.tuning.peak_sort(CA1_smoothed)
CA3_smoothed_sorted = pursuit.tuning.peak_sort(CA3_smoothed)

In [ ]:
#plot heatmaps
pursuit.tuning.heatmap(RSC_smoothed_sorted)
pursuit.tuning.heatmap(CA1_smoothed_sorted)
pursuit.tuning.heatmap(CA3_smoothed_sorted)

In [ ]:
RSC_count = pursuit.tuning.count_neurons(RSC_cleaned)
CA1_count = pursuit.tuning.count_neurons(CA1_cleaned)
CA3_count = pursuit.tuning.count_neurons(CA3_cleaned)

In [ ]:
RSC_count


In [ ]:
CA1_count

In [ ]:
CA3_count